# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, precision_score,recall_score,f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Satoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Satoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [75]:
# load data from database
engine = create_engine('sqlite:///../data/clean_dataframe.db')
df = pd.read_sql_table('clean_dataframe', engine )
X = df.message
Y = df.iloc[:,4:]
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [77]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
#     pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [78]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [80]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [81]:
pipeline.predict(X_test)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [82]:
np.array(y_test)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [83]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [84]:
y_test_df = pd.DataFrame(np.array(y_test), columns=Y.columns)

In [85]:
predict_test_df = pd.DataFrame(pipeline.predict(X_test), columns=Y.columns)

In [86]:
print(classification_report(y_test_df.request ,predict_test_df.request ))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5415
           1       0.90      0.43      0.58      1139

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554



In [87]:
for i in Y.columns:
    print(f'classification_report for {i}:')
    print(classification_report(y_test_df[i] ,predict_test_df[i] ))

classification_report for related:
              precision    recall  f1-score   support

           0       0.73      0.27      0.40      1492
           1       0.81      0.97      0.88      5020
           2       0.39      0.17      0.23        42

    accuracy                           0.80      6554
   macro avg       0.64      0.47      0.50      6554
weighted avg       0.79      0.80      0.77      6554

classification_report for request:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5415
           1       0.90      0.43      0.58      1139

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

classification_report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy             

C:\Users\Satoshi\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5792
           1       0.85      0.43      0.57       762

    accuracy                           0.93      6554
   macro avg       0.89      0.71      0.77      6554
weighted avg       0.92      0.93      0.91      6554

classification_report for shelter:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5984
           1       0.86      0.23      0.36       570

    accuracy                           0.93      6554
   macro avg       0.90      0.61      0.66      6554
weighted avg       0.92      0.93      0.91      6554

classification_report for clothing:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6450
           1       0.83      0.05      0.09       104

    accuracy                           0.98      6554
   macro avg       0.91      0.52      0.54      6554
wei

### 6. Improve your model
Use grid search to find better parameters. 

In [88]:
from sklearn.model_selection import GridSearchCV

In [89]:
parameters = {
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__max_depth':[None, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters , verbose=True , n_jobs= -1 ,cv = 3 )

cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  1.3min remaining:   15.7s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [90]:
predict_test_df = pd.DataFrame(cv.predict(X_test), columns=Y.columns)

In [91]:
for i in Y.columns:
    print(f'classification_report for {i}:')
    print(classification_report(y_test_df[i] ,predict_test_df[i] ))

classification_report for related:
              precision    recall  f1-score   support

           0       0.67      0.31      0.42      1492
           1       0.82      0.95      0.88      5020
           2       0.50      0.19      0.28        42

    accuracy                           0.80      6554
   macro avg       0.66      0.48      0.53      6554
weighted avg       0.78      0.80      0.77      6554

classification_report for request:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5415
           1       0.88      0.41      0.56      1139

    accuracy                           0.89      6554
   macro avg       0.88      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554

classification_report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy             

C:\Users\Satoshi\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6160
           1       0.84      0.23      0.37       394

    accuracy                           0.95      6554
   macro avg       0.89      0.62      0.67      6554
weighted avg       0.95      0.95      0.94      6554

classification_report for food:
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      5792
           1       0.88      0.29      0.43       762

    accuracy                           0.91      6554
   macro avg       0.90      0.64      0.69      6554
weighted avg       0.91      0.91      0.89      6554

classification_report for shelter:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5984
           1       0.84      0.21      0.33       570

    accuracy                           0.93      6554
   macro avg       0.89      0.60      0.65      6554
weighte

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [92]:
import pickle

In [93]:
# save the model to disk
filename = '../models/finalized_model.sav'
pickle.dump(cv, open(filename, 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.